In [4]:
import sqlite3, pandas
cnx = sqlite3.connect('data/QAT.db')

Question = pandas.read_sql('select * from Question', cnx)
Answer = pandas.read_sql('select * from Answer', cnx)
Tag = pandas.read_sql('select * from Tag', cnx)

In [6]:
Question[:10]

,index,Id,OwnerUserId,CreationDate,Score,Body_count,Title_count
0,0,80,26.0,2008-08-01T13:57:07Z,26,293,7
1,1,90,58.0,2008-08-01T14:41:24Z,144,24,7
2,2,120,83.0,2008-08-01T15:50:08Z,21,54,3
3,3,180,2089740.0,2008-08-01T18:42:19Z,53,41,5
4,4,260,91.0,2008-08-01T23:22:08Z,49,169,6
5,5,330,63.0,2008-08-02T02:51:36Z,29,166,8
6,6,470,71.0,2008-08-02T15:11:47Z,13,52,5
7,7,580,91.0,2008-08-02T23:30:59Z,21,246,8
8,8,650,143.0,2008-08-03T11:12:52Z,79,144,4
9,9,810,233.0,2008-08-03T20:35:01Z,9,179,9


In [18]:
Answer[:10]

,index,Id,OwnerUserId,CreationDate,ParentId,Score,Body_count
0,0,92,61.0,2008-08-01T14:45:37Z,90,13,18
1,1,124,26.0,2008-08-01T16:09:47Z,80,12,59
2,2,199,50.0,2008-08-01T19:36:46Z,180,1,279
3,3,269,91.0,2008-08-01T23:49:57Z,260,4,159
4,4,307,49.0,2008-08-02T01:49:46Z,260,28,137
5,5,332,59.0,2008-08-02T03:00:24Z,330,19,131
6,6,344,100.0,2008-08-02T04:18:15Z,260,6,73
7,7,359,119.0,2008-08-02T06:16:23Z,260,5,63
8,8,473,49.0,2008-08-02T15:33:13Z,470,8,108
9,9,529,86.0,2008-08-02T18:16:07Z,180,3,52


In [20]:
count = 0
for idx, row_data in Answer[9000:].iterrows():
    #print(row_data)
    if(row_data['ParentId'] not in Question['Id']):
        print('useless data', row_data['Id'])
        if(row_data['ParentId'] in Answer['Id']):
            print('subAnswer')
        count+=1
count

2809
subAnswer
useless data 2012825
subAnswer
useless data 2012828
subAnswer
useless data 2012831
subAnswer
useless data 2012840
subAnswer
useless data 2012863
subAnswer
useless data 2012867
subAnswer
useless data 2012868
subAnswer
useless data 2012898
subAnswer
useless data 2012941
subAnswer
useless data 2012949
subAnswer
useless data 2012963
subAnswer
useless data 2012964
subAnswer
useless data 2012966
subAnswer
useless data 2012981
subAnswer
useless data 2012999
subAnswer
useless data 2013005
subAnswer
useless data 2013006
subAnswer
useless data 2013012
subAnswer
useless data 2013016
subAnswer
useless data 2013039
subAnswer
useless data 2013044
subAnswer
useless data 2013045
subAnswer
useless data 2013054
subAnswer
useless data 2013056
subAnswer
useless data 2013057
subAnswer
useless data 2013077
subAnswer
useless data 2013095
subAnswer
useless data 2013109
subAnswer
useless data 2013127
subAnswer
useless data 2013134
subAnswer
useless data 2013220
subAnswer
useless data 2013250
sub

KeyboardInterrupt: 

In [39]:
Answer[Answer.eq('3872008').any(1)]

,index,Id,OwnerUserId,CreationDate,ParentId,Score,Body_count


In [36]:
len(Question)

1264216

In [37]:
len(Answer)

2014516

In [38]:
count = 0
for idx, row_data in Question[:].iterrows():
    #print(row_data)
    if(row_data['Id'] not in Answer['ParentId']):
        print('useless data', row_data['Id'])
        '''if(row_data['ParentId'] in Answer['Id']):
            print('subAnswer')'''
        count+=1
count

3868810
useless data 3868820
useless data 3868900
useless data 3868920
useless data 3868960
useless data 3868980
useless data 3868990
useless data 3869040
useless data 3869110
useless data 3869260
useless data 3869280
useless data 3869290
useless data 3869320
useless data 3869330
useless data 3869350
useless data 3869360
useless data 3869380
useless data 3869450
useless data 3869520
useless data 3869550
useless data 3869570
useless data 3869580
useless data 3869650
useless data 3869710
useless data 3869740
useless data 3869750
useless data 3869770
useless data 3869790
useless data 3869810
useless data 3869830
useless data 3869860
useless data 3869880
useless data 3869900
useless data 3869960
useless data 3870020
useless data 3870040
useless data 3870050
useless data 3870090
useless data 3870110
useless data 3870160
useless data 3870230
useless data 3870260
useless data 3870330
useless data 3870430
useless data 3870480
useless data 3870500
useless data 3870530
useless data 3870540
usele

KeyboardInterrupt: 